In [27]:
!pip install -q datasets
!pip install -q huggingface_hub

In [ ]:
import os
os.environ["HF_TOKEN"] = "hf_your_token_here" # Replace with your actual Hugging Face token

In [28]:
# Load the dataset from HuggingFace
from datasets import load_dataset
import random


# Download and load the dataset
dataset = load_dataset("Helsinki-NLP/europarl",'de-fr')
dataset = dataset['train'].shuffle(random.randint(0, 100)).select(i for i in range(800))

In [26]:
from huggingface_hub import InferenceClient
import random, json

client = InferenceClient(
	provider="hf-inference",
	api_key=os.environ["HF_TOKEN"],
)

how_many_new = 10
pick_samples = 5
picked = dataset.shuffle(random.randint(0, 100)).select(i for i in range(pick_samples))
prompt = f"""
You are a highly skilled AI language translator specialized in German-to-French translation.
Generate {how_many_new} unique translations from German to formal French using the below examples. Keep your translations relevant to the theme.
Output only a JSON array with objects structured as:
{json.dumps(picked['translation'][:pick_samples], indent=2)}
"""

print("Prompt:")
print(prompt)

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=messages,
    max_tokens=15000,
)

output = completion.choices[0].message
if isinstance(output, dict) and "content" in output:
    output = output["content"]

# Print the model's output (for verification)
print("Model output:")
print(output)

# Attempt to parse the output as JSON
try:
    translations = json.loads(output)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    exit(1)

# Save the JSON array to a file
with open("french_translations.json", "w", encoding="utf-8") as f:
    json.dump(translations, f, ensure_ascii=False, indent=2)

print("Translation pairs saved to french_translations.json")

Prompt:

You are a highly skilled AI language translator specialized in German-to-French translation.
Generate 10 unique translations from German to formal French using the below examples. Keep your translations relevant to the theme.
Output only a JSON array with objects structured as:
[
  {
    "de": "Die Kommission hatte gehofft, dass die Aufnahme der Beitrittsverhandlungen dem Reformprozess sp\u00fcrbarere Impulse verleihen w\u00fcrde.",
    "fr": "La Commission esp\u00e9rait que l\u2019ouverture des n\u00e9gociations d\u2019adh\u00e9sion imprimerait un \u00e9lan plus tangible au processus de r\u00e9forme."
  },
  {
    "de": "Im M\u00e4rz legte die Regierung ihr nationales Programm zur \u00dcbernahme des Besitzstandes vor.",
    "fr": "En mars, le gouvernement a pr\u00e9sent\u00e9 son programme national pour l'adoption de l'acquis."
  },
  {
    "de": "Die \u00c4u\u00dferungen von Herrn Deva \u00fcber eine st\u00e4rkere Einbindung und Mitverantwortung der gro\u00dfen multinational

In [ ]:
import json

def remove_duplicates(input_file, output_file, record_limit=1600):
    # Load the JSON data from the file
    with open(input_file, 'r', encoding='utf-8') as f:
        translations = json.load(f)

    # Use a set to track seen 'de' values and store unique items
    seen_de = set()
    unique_translations = []

    for entry in translations:
        de_text = entry.get("de")
        if de_text not in seen_de:
            seen_de.add(de_text)
            unique_translations.append(entry)

    # Limit the output to the first record_limit records
    unique_translations = unique_translations[:record_limit]

    # Write the unique translations back to a new JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(unique_translations, f, ensure_ascii=False, indent=2)

    print("Unique translation pairs saved to", output_file)

# Example usage
remove_duplicates("translation copy.json", "translations_pre-processed.json")


Unique translation pairs saved to translations_pre-processed.json
